In [1]:
import polars as pl
import deltalake
import os
import sys

In [2]:
pl.__version__

'0.20.14'

In [3]:
df1 = pl.DataFrame(
    {
        "a": ["A", "B", "C"],
        "b": [4, 5, 6],
    }
)
df1

a,b
i64,i64
1,4
2,5
3,6


In [4]:
df1.write_delta("data/delta-table")


ValueError: Schema of data does not match table schema
Data schema:
a: int64
b: int64
Table Schema:
id: int64
name: large_string

In [108]:
location = "/Users/tweber/Gits/depictio/minio_data/depictio-bucket/"
# location = f"../../{location}"
for root, dirs, files in os.walk(location):
    for file in files:
        print(file)
        print(os.path.join(root, file))
        if (file.endswith(".parquet")) and ("test_table" not in os.path.join(root, file)):
            print("parquet")
            delta_table = "/".join(os.path.join(root, file).split("/")[:-1]) + "/"
print(delta_table)
delta_table = "/Users/tweber/Gits/depictio/minio_data/depictio-bucket/65e5f007bad32df857a53cf2/65ef2b8624e251dd1fb14e9c/65ef2b8624e251dd1fb14e9d/"

0-9b9a6e2f-67ad-4cae-8f1a-fc8e236950ac-0.parquet
/Users/tweber/Gits/depictio/minio_data/depictio-bucket/65e5f007bad32df857a53cf2/65ef276f4d7931217cdc86ed/65ef276f4d7931217cdc86ee/0-9b9a6e2f-67ad-4cae-8f1a-fc8e236950ac-0.parquet
parquet
1-748aaf7b-d856-4336-bb90-91a374ff3092-0.parquet
/Users/tweber/Gits/depictio/minio_data/depictio-bucket/65e5f007bad32df857a53cf2/65ef276f4d7931217cdc86ed/65ef276f4d7931217cdc86ee/1-748aaf7b-d856-4336-bb90-91a374ff3092-0.parquet
parquet
00000000000000000001.json
/Users/tweber/Gits/depictio/minio_data/depictio-bucket/65e5f007bad32df857a53cf2/65ef276f4d7931217cdc86ed/65ef276f4d7931217cdc86ee/_delta_log/00000000000000000001.json
00000000000000000000.json
/Users/tweber/Gits/depictio/minio_data/depictio-bucket/65e5f007bad32df857a53cf2/65ef276f4d7931217cdc86ed/65ef276f4d7931217cdc86ee/_delta_log/00000000000000000000.json
2-e7b20398-c95a-426c-9d88-8f7f82af7b83-0.parquet
/Users/tweber/Gits/depictio/minio_data/depictio-bucket/65e5f007bad32df857a53cf2/65ef28b156aa3

In [109]:
!tree -h -D $location

[ 128 Mar 11 16:36]  /Users/tweber/Gits/depictio/minio_data/depictio-bucket/
├── [ 448 Mar 11 17:04]  65e5f007bad32df857a53cf2
│   ├── [  96 Mar 11 16:29]  65ef236fd70ed19559c1d739
│   │   └── [  64 Mar 11 16:29]  65ef236fd70ed19559c1d73a
│   ├── [  96 Mar 11 16:30]  65ef23aad70ed19559c1d7e3
│   │   └── [  64 Mar 11 16:30]  65ef23aad70ed19559c1d7e4
│   ├── [  96 Mar 11 16:30]  65ef23b230a2f8c6bfc86863
│   │   └── [ 128 Mar 11 16:30]  65ef23b230a2f8c6bfc86864
│   │       ├── [410K Mar 11 16:30]  0-cb3706be-c99e-4f54-aa54-5da64dc92683-0.parquet
│   │       └── [  96 Mar 11 16:30]  _delta_log
│   │           └── [5.2K Mar 11 16:30]  00000000000000000000.json
│   ├── [  96 Mar 11 16:37]  65ef253f835c3dbb196daa7e
│   │   └── [ 128 Mar 11 16:37]  65ef253f835c3dbb196daa7f
│   │       ├── [410K Mar 11 16:37]  0-7e37e5d8-c74a-4f58-ab6c-1e18ff86631c-0.parquet
│   │       └── [  96 Mar 11 16:37]  _delta_log
│   │           └── [5.2K Mar 11 16:37]  00000000000000000000.json
│   ├── [  96 Mar 11 16

In [110]:
!tree -h -D $delta_table

[ 192 Mar 11 17:04]  /Users/tweber/Gits/depictio/minio_data/depictio-bucket/65e5f007bad32df857a53cf2/65ef2b8624e251dd1fb14e9c/65ef2b8624e251dd1fb14e9d/
├── [410K Mar 11 17:04]  0-344780b7-6910-45dc-8523-7ed364d1f4f5-0.parquet
├── [410K Mar 11 17:04]  1-04b86e4c-808e-470c-b70f-c5c498bcb733-0.parquet
├── [410K Mar 11 17:04]  2-d4fc6632-4431-4bf6-b65a-983c6e2d059c-0.parquet
└── [ 160 Mar 11 17:04]  _delta_log
    ├── [5.2K Mar 11 17:04]  00000000000000000000.json
    ├── [1.8K Mar 11 17:04]  00000000000000000001.json
    └── [1.8K Mar 11 17:04]  00000000000000000002.json

2 directories, 6 files


In [107]:
with pl.Config(fmt_str_lengths=50):
    print(pl.read_delta(delta_table, version=0).select("depictio_aggregation_time"))

shape: (6_720, 1)
┌───────────────────────────┐
│ depictio_aggregation_time │
│ ---                       │
│ str                       │
╞═══════════════════════════╡
│ 2024-03-11 15:46:55       │
│ 2024-03-11 15:46:55       │
│ 2024-03-11 15:46:55       │
│ 2024-03-11 15:46:55       │
│ 2024-03-11 15:46:55       │
│ …                         │
│ 2024-03-11 15:46:55       │
│ 2024-03-11 15:46:55       │
│ 2024-03-11 15:46:55       │
│ 2024-03-11 15:46:55       │
│ 2024-03-11 15:46:55       │
└───────────────────────────┘


In [6]:
!ls -alh ../../minio_data/depictio-bucket/65e5f007bad32df857a53cf2/65ef057b6832d5ac36e3a1df/65ef057b6832d5ac36e3a1e0/_delta_log

total 0
drwxr-xr-x  4 tweber  staff   128B Mar 11 14:26 .
drwxr-xr-x  5 tweber  staff   160B Mar 11 14:26 ..
drwxr-xr-x  3 tweber  staff    96B Mar 11 14:26 00000000000000000000.json
drwxr-xr-x  3 tweber  staff    96B Mar 11 14:26 00000000000000000001.json


In [15]:
!ls ../.././minio_data/depictio-bucket/65e5f007bad32df857a53cf2/65ef0cf6ba771d68d82abe9e/65ef0cf6ba771d68d82abe9f

0-7ba3c483-ced7-47cb-ad71-01822acd5b43-0.parquet
_delta_log


In [17]:
dt = deltalake.DeltaTable("../../minio_data/depictio-bucket/65e5f007bad32df857a53cf2/65ef0cf6ba771d68d82abe9e/65ef0cf6ba771d68d82abe9f/")

TableNotFoundError: no log files

In [75]:
import pandas as pd


df = pd.DataFrame([{"id": 1, "name": "foo"}, {"id": 2, "name": "bar"}, {"id": 3, "name": "baz"}])

pl_df = pl.DataFrame(df)

pl_df.write_delta("../../minio_data/depictio-bucket/test_table/", mode="overwrite")

In [45]:
pl.read_delta("../../minio_data/depictio-bucket/test_table", version=0)

id,name
i64,str
1,"""foo"""
2,"""bar"""
3,"""baz"""


In [25]:
pl.read_delta("../.././minio_data/depictio-bucket/65e5f007bad32df857a53cf2/65ef0ef8cbb1b4b72250fb0f/65ef0ef8cbb1b4b72250fb10/")

TableNotFoundError: no log files

In [2]:
pl.__version__

'0.20.14'

In [3]:
# deltalake version
import deltalake
import pandas as pd

df = pd.DataFrame([{"id": 1, "name": "foo"}, {"id": 2, "name": "bar"}, {"id": 3, "name": "baz"}])
deltalake.write_deltalake("./data/delta-table", df, mode="overwrite")

# Load the table
table = deltalake.DeltaTable("./data/delta-table")
table.to_pandas()

,id,name
0,1,foo
1,2,bar
2,3,baz


In [6]:
import boto3

s3 = boto3.client("s3", endpoint_url="http://localhost:9000", aws_access_key_id="minio", aws_secret_access_key="minio123", region_name="us-east-1")
from pprint import pprint

pprint(s3.list_buckets())

{'Buckets': [{'CreationDate': datetime.datetime(2024, 3, 20, 18, 27, 16, 942000, tzinfo=tzutc()),
              'Name': 'depictio-bucket'}],
 'Owner': {'DisplayName': 'minio',
           'ID': '02d6176db174dc93cb1b899f7c6078f08654445fe8cf1b6ce98d8855f66bdbf4'},
 'ResponseMetadata': {'HTTPHeaders': {'accept-ranges': 'bytes',
                                      'content-length': '373',
                                      'content-type': 'application/xml',
                                      'date': 'Fri, 12 Apr 2024 12:34:05 GMT',
                                      'server': 'MinIO',
                                      'strict-transport-security': 'max-age=31536000; '
                                                                   'includeSubDomains',
                                      'vary': 'Origin, Accept-Encoding',
                                      'x-amz-id-2': 'dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8',
                                 

In [1]:
import deltalake
import pandas as pd

# DataFrame creation
df = pd.DataFrame([{"id": 1, "name": "foo"}, {"id": 2, "name": "bar"}, {"id": 3, "name": "baz"}])

# MinIO storage options
storage_options = {
    "endpoint_url": "http://localhost:9000",  # Change localhost to your MinIO server IP if necessary
    "aws_access_key_id": "minio",
    "aws_secret_access_key": "minio123",
    "use_ssl": "false",
    "AWS_REGION": "us-east-1",  # Example region, adjust as needed
    "signature_version": "s3v4",
    "AWS_ALLOW_HTTP" : "true",
    "AWS_S3_ALLOW_UNSAFE_RENAME": "true"
}

# Write to MinIO
deltalake.write_deltalake("s3://depictio-bucket/data/delta-table4", df, storage_options=storage_options, mode="overwrite")

# # Load the table from MinIO
# table = deltalake.DeltaTable("s3://your-bucket-name/data/delta-table", storage_options=storage_options)
# df_from_minio = table.to_pandas()

In [2]:
table = deltalake.DeltaTable("s3://depictio-bucket/data/delta-table4", storage_options=storage_options)
df_from_minio = table.to_pandas()
df_from_minio

,id,name
0,1,foo
1,2,bar
2,3,baz


In [25]:
# do the same with polars
import polars as pl


df = pd.DataFrame([{"id": 1, "name": "foo"}, {"id": 2, "name": "bar"}, {"id": 3, "name": "baz"}, {"id": 4, "name": "baz"}])
df = pl.DataFrame(df)
df.write_delta("s3://depictio-bucket/data/delta-table5", storage_options=storage_options, mode="overwrite")
df

id,name
i64,str
1,"""foo"""
2,"""bar"""
3,"""baz"""
4,"""baz"""


In [15]:
import boto3
from botocore.exceptions import NoCredentialsError

# MinIO credentials
minio_endpoint = "http://localhost:9000"  # Change to your MinIO server URL
access_key = "minio"
secret_key = "minio123"
bucket_name = "depictio-bucket"
destination_prefix = "67a25c82fea6466823de362"  # Remove 's3://' when using boto3

storage_options = {
    "endpoint_url": minio_endpoint,
    "aws_access_key_id": access_key,
    "aws_secret_access_key": secret_key,
    "use_ssl": "false",
    "AWS_REGION": "us-east-1",
    "signature_version": "s3v4",
    "AWS_ALLOW_HTTP": "true",
    "AWS_S3_ALLOW_UNSAFE_RENAME": "true"
}

import deltalake.exceptions
import polars as pl
import deltalake

try:
    df = pl.read_delta(f"s3://{bucket_name}/{destination_prefix}", storage_options=storage_options)
except deltalake.exceptions.TableNotFoundError as e:
    print("Table not found:", e)
    # Handle the error or provide a fallback
    # For example, you could create a new Delta table or log the error


Table not found: no log files


In [2]:
# read using polars from minio
# list versions of the table in the delta table

import polars as pl
storage_options = {
    "endpoint_url": "http://localhost:9000",  # Change localhost to your MinIO server IP if necessary
    "aws_access_key_id": "minio",
    "aws_secret_access_key": "minio123",
    "use_ssl": "false",
    "AWS_REGION": "us-east-1",  # Example region, adjust as needed
    "signature_version": "s3v4",
    "AWS_ALLOW_HTTP" : "true",
    "AWS_S3_ALLOW_UNSAFE_RENAME": "true"
}
df = pl.scan_delta("s3://depictio-bucket/662acab447d998cd889678c1/6647d03b57d1fae8d89dce8d/6647d03b57d1fae8d89dce8e/", storage_options=storage_options).collect()
df


depictio_run_id,sample,cell,medbin,mapped,suppl,dupl,mapq,read2,good,pass1,nb_p,nb_r,nb_a,bam,depictio_aggregation_time
str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,str,str
"""2019-01-02-HGN…","""SSC11163x02""","""SSC11163x02PE2…",0,15308,16,3128,1346,5418,5400,0,0.0,0.0,0.0,"""/scratch/twebe…","""2024-05-17 22:…"
"""2019-01-02-HGN…","""SSC11163x02""","""SSC11163x02PE2…",28,2100880,1104,1132247,123311,421480,422738,1,0.0964147,3.11595,0.1,"""/scratch/twebe…","""2024-05-17 22:…"
"""2019-01-02-HGN…","""SSC11163x02""","""SSC11163x02PE2…",28,2100003,1006,1140881,121505,417608,419003,1,0.0964147,3.08988,0.1,"""/scratch/twebe…","""2024-05-17 22:…"
"""2019-01-02-HGN…","""SSC11163x02""","""SSC11163x02PE2…",9,709177,408,377305,37072,147010,147382,1,0.0964147,1.08703,0.1,"""/scratch/twebe…","""2024-05-17 22:…"
"""2019-01-02-HGN…","""SSC11163x02""","""SSC11163x02PE2…",8,640565,406,340904,37468,130697,131090,1,0.0964147,0.96424,0.1,"""/scratch/twebe…","""2024-05-17 22:…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2019-12-16-HWV…","""HG002x02""","""HG002x02PE2049…",27,2800954,2025,1829471,166494,400701,402263,1,0.11942,3.80519,0.1,"""/scratch/twebe…","""2024-05-17 22:…"
"""2019-12-16-HWV…","""HG002x02""","""HG002x02PE2049…",143,5013430,2716,440385,361227,2101615,2107487,1,0.11942,20.0163,0.1,"""/scratch/twebe…","""2024-05-17 22:…"
"""2019-12-16-HWV…","""HG002x02""","""HG002x02PE2049…",26,3107189,2636,2126436,172098,402015,404004,1,0.11942,3.82411,0.1,"""/scratch/twebe…","""2024-05-17 22:…"


In [10]:
pl.__version__

'0.20.14'

In [9]:
type(df["cell"].unique())

polars.series.series.Series

In [11]:
list(df["cell"].unique())

['SSC1163101PE20415',
 'HG04217x02PE20361',
 'HG00864x02PE20326',
 'BoM662Gx01PE20528',
 'GM19434x02PE20586',
 'NLEx02PE20336',
 'ckAML02x01PE20381',
 'BOM71Gx03PE20370',
 'WCHSPCCB02x01PE20586',
 'BoM69Nx01PE20480',
 'CB0003x01PE20310',
 'M56x1x02PE20446',
 'BOM65G03x03PE20412',
 'ADCr11xGFPxmCherryPE20362',
 'BoM66Gx01PE20345',
 'BoM66Gx01PE20312',
 'R8x28PE20580',
 'GM18534Bx02PE20303',
 'LFSL41midx01PE20522',
 'R8x28PE20556',
 'BoM92Nx02PE20414',
 'HG02818x02PE20369',
 'CB0003x01PE20388',
 'CAST1D1x02PE20481',
 'BoM712Gx01PE20458',
 'BoM712Gx01PE20415',
 'HG01573x02PE20387',
 'HG03065x02PE20534',
 'HG03009x02PE20366',
 'HG03065x02PE20588',
 'R8x28PE20537',
 'CB0003x01PE20391',
 'HG04217x02PE20352',
 'CAST1D1x02PE20493',
 'GM19983x02PE20455',
 'HG00171Ax02PE20470',
 'NLEx02PE20321',
 'BOMCHIP76Nx02PE20406',
 'BoM47Nx01PE20563',
 'HG01505x01PE20432',
 'BoM63Gx02PE20571',
 'GM18534Bx02PE20321',
 'BoM69Nx01PE20457',
 'BoM70Nx01PE20437',
 'HG04217x02PE20386',
 'BoM712Gx01PE20454',
 'ckA

In [4]:
import polars as pl

# Function to add filter criteria to a list
def add_filter(filter_list, interactive_component_type, column_name, value, min_value=None, max_value=None):
    print(filter_list)
    if interactive_component_type in ["Select", "MultiSelect", "SegmentedControl"]:
        if value:
            filter_list.append(pl.col(column_name).is_in(value))
    
    elif interactive_component_type == "TextInput":
        if value:
            filter_list.append(pl.col(column_name).str.contains(value))
    
    elif interactive_component_type == "Slider":
        if value:
            filter_list.append(pl.col(column_name) == value)
    
    elif interactive_component_type == "RangeSlider":
        if value:
            filter_list.append((pl.col(column_name) >= value[0]) & (pl.col(column_name) <= value[1]))

# Function to apply the cumulative filter to the DataFrame
def apply_filters(df, filter_list):
    if filter_list:
        combined_filter = filter_list[0]
        for filt in filter_list[1:]:
            combined_filter &= filt
        return df.filter(combined_filter)
    return df

# Example usage
filter_list = []

# Example interactive components and values
interactive_components = [
    {"type": "Select", "column": "category", "value": ["A", "B"]},
    {"type": "TextInput", "column": "name", "value": "John"},
    {"type": "Slider", "column": "age", "value": 30},
    {"type": "RangeSlider", "column": "salary", "value": [50000, 100000]},
]

# Populate the filter list
for component in interactive_components:
    add_filter(
        filter_list,
        interactive_component_type=component["type"],
        column_name=component["column"],
        value=component["value"]
    )

# Sample DataFrame
df = pl.DataFrame({
    "category": ["A", "B", "C", "A"],
    "name": ["John", "Jane", "John", "Doe"],
    "age": [30, 25, 40, 30],
    "salary": [60000, 80000, 70000, 90000]
})

# Apply the filters
filtered_df = apply_filters(df, filter_list)

print(filtered_df)

[]
[<Expr ['col("category").is_in([Series]…'] at 0x13278E990>]
[<Expr ['col("category").is_in([Series]…'] at 0x13278E990>, <Expr ['col("name").str.contains([Stri…'] at 0x13208ABD0>]
[<Expr ['col("category").is_in([Series]…'] at 0x13278E990>, <Expr ['col("name").str.contains([Stri…'] at 0x13208ABD0>, <Expr ['[(col("age")) == (30)]'] at 0x132F88F90>]
shape: (1, 4)
┌──────────┬──────┬─────┬────────┐
│ category ┆ name ┆ age ┆ salary │
│ ---      ┆ ---  ┆ --- ┆ ---    │
│ str      ┆ str  ┆ i64 ┆ i64    │
╞══════════╪══════╪═════╪════════╡
│ A        ┆ John ┆ 30  ┆ 60000  │
└──────────┴──────┴─────┴────────┘
